# Real world example

### Import all libs

In [2]:
import sys
import os
import torch
from datasets import load_dataset
from typing import List

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from preprocessing.transform.dataset_builder import Builder
from preprocessing.transform.tensor_dataset import TensorIterableDataset

### Create the GiftEval loader

In [13]:
class GiftEvalIterableDataset(TensorIterableDataset):
    def __init__(self, repo_id="Salesforce/GiftEvalPretrain", split="train", streaming=True, ts_size=128):
        super().__init__()
        self.repo_id = repo_id
        self.split = split
        self.streaming = streaming
        self.ts_size = ts_size
        
        self.test_example = None
        self.test_row = None

    def __iter__(self):
        dataset = iter(load_dataset(self.repo_id, split=self.split, streaming=self.streaming))
        while True:
            try:
                example = next(dataset)
                
                # to avoid the problems with different shapes of data we only take target into account
                # and return on target line at a time
                
                for row in example["target"]:
                    # to avoid different shapes problem
                        
                    for i in range(len(row) // self.ts_size):
                        left = i * self.ts_size
                        right = (i + 1) * self.ts_size
                        
                        if right >= len(row):
                            break
                            
                        ts = row[i * self.ts_size : (i + 1) * self.ts_size]
                        
                        result = torch.unsqueeze(
                            torch.tensor(ts, dtype=torch.float32), 
                            0
                        )
                        
                        yield torch.nan_to_num(result, nan=0)
            except StopIteration:
                return
            except Exception as e:
                print(f"Skipping example due to error: {e}")
                self.test_example = example
                continue

### Create the raw dataset instance

In [14]:
ds_raw = GiftEvalIterableDataset()

### Apply post-processing

In [15]:
import random

def change_random_item_to_avg(batch):
    mean = torch.stack(batch, dim=0).mean(dim=0)
    
    index = rand_index = random.randint(0, len(batch) - 1)
    batch[index] = mean
    
    result = []
    for i in range(len(batch)):
        if i == index:
            result.append(mean)
        else:
            result.append(batch[i])
    
    return result

def z_transform_tensor_list(tensor_list, eps=1e-8):
    # Stack all tensors to compute stats
    stacked = torch.cat(tensor_list, dim=1)  # shape: (num_features, total_time)

    # Compute mean and std per feature (dim=1 is time, so reduce across it)
    mean = stacked.mean(dim=1, keepdim=True)  # shape: (num_features, 1)
    std = stacked.std(dim=1, keepdim=True) + eps  # avoid division by zero

    # Normalize each tensor
    normalized_list = [(t - mean) / std for t in tensor_list]

    return normalized_list

ds = (
    Builder(ds_raw)
    .batch(10)
    .map(change_random_item_to_avg)
    .map(z_transform_tensor_list)
    .flat()
    .build()
)

In [16]:
import time
from itertools import islice

start_time = time.time()

for i, item in enumerate(islice(ds, 10_000)):
    if i % 100 == 0:
        print(i)
    
end_time = time.time()

print(f"\nElapsed time: {end_time - start_time:.2f} seconds")

Resolving data files:   0%|          | 0/6528 [00:00<?, ?it/s]

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
Skipping example due to error: Couldn't cast array of type
list<item: float>
to
List(List(Value('float32')), length=2)
9500

Elapsed time: 14.83 seconds
